# Agent playground
![image.png](https://mintcdn.com/langchain-5e9cc07a/-_xGPoyjhyiDWTPJ/oss/images/agent.png?w=840&fit=max&auto=format&n=-_xGPoyjhyiDWTPJ&q=85&s=bd932835b919f5e58be77221b6d0f194)

In [ ]:
from os import environ
environ['CUDA_VISIBLE_DEVICES'] = '1'

In [ ]:
!pip install langchain langchain_openai langchain_tavily

# Initialize the base language model

In [ ]:
local_inference = True

### A) Cloud inference 
1. via [*Hugging Face’s Inference Providers*](https://huggingface.co/docs/inference-providers/en/index)
    - Create an account for the Hugging Face platform: [huggingface.co/join](https://huggingface.co/join)
    - Get the API key from dashboard: [huggingface.co/docs/hub/en/security-tokens](https://huggingface.co/docs/hub/en/security-tokens)
2. via [*OpenAI API*](https://auth.openai.com)
    - Create a new OpenAI account (free credits): [https://auth.openai.com/log-in](https://auth.openai.com/log-in)
    - Generate the API key from the dashboard: [platform.openai.com/api-keys](https://platform.openai.com/api-keys)

In [ ]:
from os import environ
environ["HF_TOKEN"] = ""
environ["OPENAI_API_KEY"] = ""

In [ ]:
from langchain_openai import ChatOpenAI

# Cloud inference via OpenAI
if not local_inference and environ.get('OPENAI_API_KEY'):
    llm_model = ChatOpenAI(model="gpt-5-nano", api_key=environ["OPENAI_API_KEY"])
    
    print(f"Cloud inference ({llm_model.openai_api_base}): model: \"{llm_model.model_name}\"")
    
# Cloud inference via HuggingFace
elif not local_inference and environ.get('HF_TOKEN'):
    llm_model = ChatOpenAI(
        base_url="https://router.huggingface.co/v1",
        model="Qwen/Qwen3-Next-80B-A3B-Instruct", # (1) Qwen/Qwen3-Next-80B-A3B-Instruct || (2) openai/gpt-oss-120b
        api_key=environ["HF_TOKEN"])
    
    print(f"Cloud inference ({llm_model.openai_api_base}): model: \"{llm_model.model_name}\"")

### B) Local inference

In [ ]:
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace

if local_inference:
    llm_model = ChatHuggingFace(
        llm = HuggingFacePipeline.from_model_id(
            model_id="allenai/OLMo-2-0425-1B-Instruct", #  Qwen/Qwen3-4B-Instruct-2507
            task ="text-generation",
            pipeline_kwargs={'dtype':"bfloat16"}
        ))
    print(f"Local Inference: \"{llm_model.llm.model_id}\"")

# Initialize the tool
This code demonstrates how to use the `TavilySearch` tool from the `langchain_tavily` package to perform a web search within a LangChain workflow. 
1. It imports the TavilySearch class, which is a tool designed to query the Tavily Search API and return structured search results, such as URLs, snippets, and optionally images or answers.
2. The invoke method is then called with the query `"What is Italy’s current public debt?"`. 
    - This method sends the query to the Tavily API and returns the search results as a dictionary containing information such as the original query, a list of result items (with titles, URLs, and content snippets), and possibly other metadata.
3. The results are printed to the output pane, allowing you to inspect the returned data. 

The code also shows how to organize tools for later use by placing the search tool into a list called tools. This is useful when building more complex agent workflows that may use multiple tools for different tasks.

In [ ]:
from langchain_tavily import TavilySearch
from json import dumps

# Initialize the Tavily Search tool
search_tool = TavilySearch(max_results=2, tavily_api_key = "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")

# Try out the search tool
#search_results = search_tool.invoke(input = "What is Italy’s current public debt?")
#print(dumps(search_results, indent=4))

# Invoke the agent with a user query


In [ ]:
query = "What's the weather like today in Trento, Italy?"

### A) without the search tool

In [ ]:
from langchain.agents import create_agent
agent_executor = create_agent(
    model = llm_model)
    #system_prompt = "You are a helpful assistant that exploits all available tools to find up-to-date information.")

In [ ]:
# Define the input message
input_message = {"messages": {"role": "user", "content": query}}

# Invoke the agent
response = agent_executor.invoke(input_message)

# Print the response
for message in response["messages"]:
    message.pretty_print()

### B) with the search tool

In [ ]:
agent_executor = create_agent(
    model = llm_model, 
    tools = [search_tool],
    system_prompt = "You are a helpful assistant that exploits all available tools to find up-to-date information.")

In [ ]:
# Define the input message
input_message = {"messages": {"role": "user", "content": query}}

# Invoke the agent
response = agent_executor.invoke(input_message)

# Print the response
for message in response["messages"]:
    message.pretty_print()

# Create our custom tools

In [ ]:
def get_exam_score(exam_name: str) -> dict:
    """Get the expected score for a given exam."""
    
    # For demonstration purposes, we assume a perfect score (we have high expectations!)
    student_score = 30
    
    return {
        'exam_name': exam_name, 
        'range': (0, 30), 
        'score': student_score}

def parse_result(score: int) -> dict:
    """Get the expected result (pass or fail) for a given score."""
    
    # For demonstration purposes, we assume a traditional passing threshold
    pass_threshold = 18
    
    # Context: exam is graded out of 30, with 18 as the passing threshold
    results = {
        'score': score, 
        'pass_threshold': pass_threshold,
        'passed': score >= pass_threshold,
        'cum_laude': False # sorry :/
    }
    
    return results

In [ ]:
agent_executor = create_agent(
    model = llm_model, 
    tools = [get_exam_score, parse_result],
    system_prompt = "You are a helpful assistant that exploits all available tools to find up-to-date information.")

In [ ]:
# Wait for 5 seconds to avoid rate limiting issues
import time 
time.sleep(10)

In [ ]:
query = 'Will I ever pass the FM 2025 exam?'

In [ ]:
response = agent_executor.invoke({"messages": {"role": "user", "content": query}})

# Print the response
for message in response["messages"]:
    message.pretty_print()

# Human in the loop

In [ ]:
def parse_result(score: int) -> dict:
    """Get the expected result (pass or fail) for a given score."""
    
    # For demonstration purposes, we assume a traditional passing threshold
    pass_threshold = 18
    
    # Ask for human approval if the score is passing
    accepted = None
    if score >= pass_threshold:
        user_input = input(f"Do you accept a score equal to {score} (yes/no): ").strip().lower()
        accepted = True if user_input == 'yes' else False
    
    # Context: exam is graded out of 30, with 18 as the passing threshold
    results = {
        'score': score, 
        'pass_threshold': pass_threshold,
        'passed': score >= pass_threshold,
        'cum_laude': False, # sorry :/
        'acceptedByStudent': accepted
    }
    
    return results

In [ ]:
agent_executor = create_agent(
    model = llm_model, 
    tools = [get_exam_score, parse_result],
    system_prompt = "You are a helpful assistant that exploits all available tools to find up-to-date information.")

In [ ]:
response = agent_executor.invoke({"messages": {"role": "user", "content": query}}, config = {"configurable": {"thread_id": "101"}})

# Print the response
for message in response["messages"]:
    message.pretty_print()

# Conversetional agents (i.e., chat bot)

In [ ]:
from langchain_community.chat_message_histories.in_memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables import RunnableMap
import time

In [ ]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """Retrieve or create chat history for a session."""
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

def handle_command(user_input: str, session_id: str) -> tuple[str, str]:
    """Handle special commands and return (response, new_session_id)."""
    if user_input.startswith("/clear"):
        store[session_id] = ChatMessageHistory()  # Reset history
        return "Chat history cleared.", session_id
    elif user_input.startswith("/session "):
        new_session = user_input.split(" ", 1)[1].strip()
        return f"Switched to session '{new_session}'.", new_session
    elif user_input == "/help":
        return ("Commands: /clear (reset history), /session <id> (switch session), /exit (quit). Otherwise, just chat!", session_id)
    return None, session_id  # Not a command

In [ ]:
# Define the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Respond naturally and assist with queries."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessage(content="{input}")##("human", "{input}")
])

# Chain the prompt with the LLM
chain = prompt | llm_model

# Create the chatbot with history support
chatbot = RunnableWithMessageHistory(chain, get_session_history=get_session_history, input_messages_key="input", history_messages_key="history")

print("Chatbot ready! Type '/help' for commands or 'exit' to quit.")
session_id = "user1"

while True:
    try:
        user_input = input("You: ").strip()
        if not user_input:
            print("Bot: Please enter a message.")
            continue
        
        # Handle special commands
        command_response, session_id = handle_command(user_input, session_id)
        if command_response:
            print(f"Bot: {command_response}")
            continue
        
        # Exit condition
        if user_input.lower() in ["exit", "quit"]:
            print("Bot: Goodbye!")
            break
        
        # Invoke the chatbot
        result = chatbot.invoke(input = {"input": user_input}, config={"configurable": {"session_id": session_id}})
        
        # Print response with a separator
        print(f"Bot ({session_id}): {result.content}")
        time.sleep(0.5)  # Small delay for natural feel
    
    except Exception as e:
        print(f"Error: {str(e)}. Please try again.")
        continue

In [ ]:
def chat_with_ai(user_input, history=[]):
    response = chatbot.invoke(input = {"input": user_input}, config={"configurable": {"session_id": session_id}})
    return response.content

In [ ]:
!pip install gradio

In [ ]:
def respond(message, history, session_id):
    """Handles a single user message and updates chat history."""
    response = chatbot.invoke(input = {"input": user_input}, config={"configurable": {"session_id": session_id}})
    return response.content

In [ ]:
import gradio as gr

# Create the Gradio chat interface
interface = gr.ChatInterface(
    fn=respond,
    title="LangChain 1.0 Chatbot 🤖",
    description="A memory-aware chatbot built with LangChain 1.0 + OpenAI + Gradio.",
    examples=[
        ["Hello!", "user1"],
        ["Tell me a joke about cats.", "user1"],
        ["What's the capital of Japan?", "user2"]
    ],
    additional_inputs=[
        gr.Textbox(label="Session ID", value="user1", info="Set a unique session name to keep context.")
    ])

# Launch the Gradio interface with sharing enabled
interface.launch(share=True)